In [1]:
cd ..

/Users/deepakduggirala/Documents/autonomous-robotics/bev-generation


In [2]:
import time
import zlib
from pathlib import Path
from itertools import islice

import numpy as np

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

import lmdb

from tqdm import tqdm

import utils
from utils import array_to_bytes, bytes_to_array
import nusc_utils

In [5]:
def batched(iterable, n):
    """Batch data into lists of length n. The last batch may be shorter."""
    # batched('ABCDEFG', 3) --> ABC DEF G
    it = iter(iterable)
    while True:
        batch = list(islice(it, n))
        if not batch:
            return
        yield batch

In [6]:
def read_bev(bev_top_filepath):
    return np.load(bev_top_filepath, 'r')['arr_0']

def transform_bev(bev_top):

    for cls_i in np.unique(bev_top):
        ii, jj = np.where(bev_top == cls_i)
        cls_j = classes_remap.get(cls_i, 0)
        bev_top[ii, jj] = cls_j
    return bev_top

In [25]:
def process_batch(lmdb_env, batch):
    with lmdb_env.begin(write=True) as write_txn:
        for bev_top_filepath in batch:
            bev_top = read_bev(str(bev_top_filepath))
            bev_top_remapped = transform_bev(bev_top)
            
            key = bytes(bev_top_filepath.stem, 'utf-8')
            value = array_to_bytes(bev_top_remapped)
            value_zipped = zlib.compress(value)
            write_txn.put(key, value_zipped)
            
def process_bev(db_path, bev_top_dir):
    db_path.mkdir(parents=True, exist_ok=True)
    db_map_size = int(15 * 1024 * 1024 * 1024)
    with lmdb.open(path=str(db_path), map_size=db_map_size) as lmdb_env:
        for i, batch in tqdm(enumerate(batched(bev_top_dir.iterdir(), BATCH_SIZE), 1)):
            # print('starting batch', i)
            # start = time.perf_counter()
            process_batch(lmdb_env, batch)
            # print('completed batch', i, 'time took', time.perf_counter() - start)

In [32]:
BATCH_SIZE=50

vehicle_id=5
bev_top_dir = dataroot / f'sweeps/BEV_TOP_id_{vehicle_id}'
db_path = dataroot / Path(f'lmdb/samples/GT_BEV_id_{vehicle_id}')

process_bev(db_path, bev_top_dir)

46it [00:09,  4.93it/s]


In [4]:
dataroot = Path('/Users/deepakduggirala/Documents/autonomous-robotics/V2X-Sim-2.0/').resolve()
db_path = dataroot / Path(f'lmdb/samples/GT_BEV')
db_path.mkdir(parents=True, exist_ok=True)
db_map_size = int(15 * 1024 * 1024 * 1024)

with lmdb.open(path=str(db_path), map_size=db_map_size) as lmdb_env:
    for vehicle_id in range(1,6):
        print('processing', vehicle_id)
        gt_db_path = dataroot / Path(f'lmdb/samples/GT_BEV_id_{vehicle_id}')
        with lmdb.open(path=str(gt_db_path), readonly=True) as gt_db:
            with gt_db.begin() as read_txn:
                cursor = read_txn.cursor()
                with lmdb_env.begin(write=True) as write_txn:
                    for key, value in cursor:
                        key_str = bytes.decode(key, 'utf-8')
                        new_key = f'${key_str}_id_{vehicle_id}'
                        new_key_bytes = bytes(new_key, 'utf-8')
                        write_txn.put(new_key_bytes, value)

processing 1
processing 2
processing 3
processing 4
processing 5
